In [1]:
# default libs
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.3f}'.format)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# plotting libs

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import datashader as ds
import colorcet as cc

# ML lib
        
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

---
# Loading and feature engineering

## Loading

We load all three datasets and put the airport info into the `flights` sets.

In [3]:
airports = pd.read_csv('/kaggle/input/flight-delays-prediction-challeng2021/airports.csv')
flights_train = pd.read_csv('/kaggle/input/flight-delays-prediction-challeng2021/flights_train.csv')
flights_test = pd.read_csv('/kaggle/input/flight-delays-prediction-challeng2021/flights_test.csv')

airports_origin = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'ORIGIN_AIRPORT'})
airports_arrive = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'DESTINATION_AIRPORT'})

# make airports nice
flights_train = flights_train.merge(airports_origin, on = 'ORIGIN_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_origin', 'LONGITUDE' : 'LONGITUDE_origin'})
flights_train = flights_train.merge(airports_arrive, on = 'DESTINATION_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_arrival', 'LONGITUDE' : 'LONGITUDE_arrival'})

# make airports nice
flights_test = flights_test.merge(airports_origin, on = 'ORIGIN_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_origin', 'LONGITUDE' : 'LONGITUDE_origin'})
flights_test = flights_test.merge(airports_arrive, on = 'DESTINATION_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_arrival', 'LONGITUDE' : 'LONGITUDE_arrival'})


In [4]:
airports = flights_train.ORIGIN_AIRPORT.unique().tolist()
airports.extend(flights_train.DESTINATION_AIRPORT.unique())
airports.extend(flights_train.ORIGIN_AIRPORT.unique())
airports.extend(flights_train.DESTINATION_AIRPORT.unique())

airports = list(set(airports))

### and load weather data v1

In [ ]:
if False :
    weather = pd.DataFrame(columns=[
    "station",
    "valid",
    "tmpc",
    "sknt",
    "p01m",
    "vsby",
    "gust",
    "skyc1",
    "skyc2",
    "skyc3",
    "wxcodes",
    "ice_accretion_6hr",
    "snowdepth"
    ])
    for code in airports:
        url = f"https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={code}&data=tmpc&data=sknt&data=p01m&data=vsby&data=gust&data=skyc1&data=skyc2&data=skyc3&data=wxcodes&data=ice_accretion_6hr&data=snowdepth&year1=2015&month1=1&day1=1&year2=2015&month2=8&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=T&direct=no&report_type=1&report_type=2"
        df = pd.read_csv(url)
        weather = weather.append(df)
    
    weather.station.unique().shape[0]

### load 2015 more data pls

In [5]:
extra = pd.read_csv('../input/flightpredictiont35/2015.csv')
extra = extra[extra.FL_DATE < '2015-08-01']
extra = extra.drop(columns = [
    "Unnamed: 27"
    ])

In [6]:
extra["MATCH"] = extra.FL_DATE.astype(str) + extra.OP_CARRIER + extra.OP_CARRIER_FL_NUM.astype(str) + extra.ORIGIN + extra.DEST

In [7]:
flights_train["MATCH"] = pd.to_datetime(flights_train[['YEAR', 'MONTH', 'DAY']])
flights_train.MATCH = flights_train.MATCH.astype(str) + flights_train.AIRLINE + flights_train.FLIGHT_NUMBER.astype(str) + flights_train.ORIGIN_AIRPORT + flights_train.DESTINATION_AIRPORT

In [8]:
flights_test["MATCH"] = pd.to_datetime(flights_test[['YEAR', 'MONTH', 'DAY']])
flights_test.MATCH = flights_test.MATCH.astype(str) + flights_test.AIRLINE + flights_test.FLIGHT_NUMBER.astype(str) + flights_test.ORIGIN_AIRPORT + flights_test.DESTINATION_AIRPORT

In [ ]:
extra = extra[["MATCH", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY", "ARR_DELAY"]]
flights_test_plus = flights_test.merge(extra, how='left', on='MATCH')
extra = extra[["MATCH", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]]
flights_train_plus = flights_train.merge(extra, how='left', on='MATCH')

In [ ]:
flights_train_plus = flights_train_plus.fillna(0)
flights_test_plus = flights_test_plus.fillna(0)

## capturing important info

In this section we gather some information about the data set we think is important for feature engineering :)

### airport and airline rankings

There are a TON of airports, they are computationally intensive if we want to include the arrival and destinations as categories. Keeping the top something and one hot encoding those is an option. but what might be nicer (and less intensive) is ranking them by their average delay, and feeding that rank into the model. 

In [ ]:
orig_rank = flights_train.groupby('ORIGIN_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
dest_rank = flights_train.groupby('DESTINATION_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
airl_rank = flights_train.groupby('AIRLINE').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
orig_rank = {p:i for i, p in enumerate(orig_rank)}
dest_rank = {p:i for i, p in enumerate(dest_rank)}
airl_rank = {p:i for i, p in enumerate(airl_rank)}

### different time columns

In [ ]:
clock_time_columns = [
    'SCHEDULED_DEPARTURE', # planned leaving time
    'DEPARTURE_TIME',      # actual leaving time
    'WHEELS_OFF',          # moment the airplane leaves the ground
    'SCHEDULED_ARRIVAL',   # planned arrical
]
relative_time_columns = [
    'TAXI_OUT',            # mintues between departure and wheels off
    'SCHEDULED_TIME',      # planned flight (+ground?) time
    'ARRIVAL_DELAY'        # difference between scheduled arrival and actual arrival
]

### outliers

harms performance

In [ ]:
stds = 4
# calculate summary statistics
data_mean, data_std = flights_train["ARRIVAL_DELAY"].mean(), flights_train["ARRIVAL_DELAY"].std()
# identify outliers
cut_off = data_std * stds
lower, upper = data_mean - cut_off, data_mean + cut_off

x_upper = flights_train[flights_train["ARRIVAL_DELAY"] > upper]
x_lower = flights_train[flights_train["ARRIVAL_DELAY"] < lower]
total = flights_train.shape[0]

print("\n")
print(f'There are {x_upper.shape[0]} ({(x_upper.shape[0]/total*100):.2f}%) observations above {upper:.2f} ({stds} standard deviations).')
print(f'There are {x_lower.shape[0]} ({(x_lower.shape[0]/total*100):.2f}%) observations below {lower:.2f} ({stds} standard deviations).')

## Data prep

In [ ]:
def prep_df(df_in, rm_outliers = False):

    # drop useless columns
    out = df_in.drop(columns=[
        # O/A airports and distance are known
        'LATITUDE_origin',
        'LONGITUDE_origin',
        'LATITUDE_arrival',
        'LONGITUDE_arrival',
        # most date stuff is useless
        'YEAR', # only 2015
        # 'MONTH', # no repetition with test
        # probably useless columns
        'FLIGHT_NUMBER', # garbage
        'TAIL_NUMBER', # also uninteresting


    ])

    for x in out.columns:
        # set float columns to int
        if (out[x].dtype == 'float64'):
            out[x] = out[x].astype(np.int64)
        # fix clock time columns lol
        if x in clock_time_columns:
            out[x] = ((out[x] // 100)*60) + (out[x] % 100)

#     # did it carry over days - useless
#     out["CROSS_DAY"] = np.where(out.SCHEDULED_DEPARTURE > out.SCHEDULED_ARRIVAL, 1, 0)
#     out["CROSS_DAY"] = out.CROSS_DAY.astype(np.bool_)
#     out["MULTI_DAY"] = np.where(out.SCHEDULED_TIME > 1440, 1, 0)
#     out["MULTI_DAY"] = out.MULTI_DAY.astype(np.bool_)

    # no account for days
    # df_out.insert(10, "DEPARTURE_DELAY", (df_out["DEPARTURE_TIME"] - df_out["SCHEDULED_DEPARTURE"]))
    # yes account for days but break some things on the way, still better than w/o probably
    out["DEPARTURE_DELAY"] = out.DEPARTURE_TIME - out.SCHEDULED_DEPARTURE
    # very few planes leave more than 1.5 hr early
    out["DEPARTURE_DELAY_COR"] = np.where(out.DEPARTURE_DELAY < -90, out.DEPARTURE_DELAY + 1440, out.DEPARTURE_DELAY)
    
    # should help with reconstructing days,
    out["TAXI_OUT2"]       = out.DEPARTURE_TIME      - out.WHEELS_OFF
    out["TAXI_OUT_COMP"]   = out.TAXI_OUT            - out.TAXI_OUT2
    out["TOTALTIME_LEFT"]  = out.DEPARTURE_TIME      - out.SCHEDULED_ARRIVAL
    out["TAKEOFF_DELAY"]   = out.SCHEDULED_DEPARTURE - out.WHEELS_OFF
    out["SCHEDULED_TIME2"] = out.SCHEDULED_DEPARTURE - out.SCHEDULED_ARRIVAL
    out["AIRTIME_LEFT"]    = out.WHEELS_OFF          - out.SCHEDULED_ARRIVAL

    # airport destination and arrival rank
    out.insert(3, "O_RANK", out["ORIGIN_AIRPORT"].map(orig_rank))
    out.insert(3, "D_RANK", out["DESTINATION_AIRPORT"].map(dest_rank))
    out.insert(3, "A_RANK", out["AIRLINE"].map(airl_rank))
    

    out = out.drop(columns = [
        "ORIGIN_AIRPORT",
        "DESTINATION_AIRPORT",
        'MATCH'
    ])
    
    if(rm_outliers):
        out = out[out["ARRIVAL_DELAY"] < upper]
        out = out[out["ARRIVAL_DELAY"] > lower]
    return out

In [10]:
ptrain = prep_df(flights_train_plus, rm_outliers = False)
ptrain.sort_values(by="id")

In [32]:
ptest = prep_df(flights_test_plus)
ptest.sort_values(by="id")

In [33]:
del extra
del flights_train
del flights_test
del flights_train_plus
del flights_test_plus

In [ ]:
graph = sns.scatterplot(data=ptrain, x="ARRIVAL_DELAY", y="DEPARTURE_DELAY2", hue="TAXI_OUT")
graph.axhline(-90)
graph

In [2]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
ax = sns.scatterplot(data=ptrain, y="ARRIVAL_DELAY", x="DEPARTURE_DELAY_COR", color='k')
ax.set_title('Arrival delay vs Corrected departure delay')
ax.set_ylabel('Arrival delay')
ax.set_xlabel('Corrected departure delay')

plt.show()


In [ ]:
cvs = ds.Canvas(plot_width=1000, plot_height=1000)  # auto range or provide the `bounds` argument
agg = cvs.points(ptrain, 'WEATHER_DELAY', 'ARRIVAL_DELAY')  # this is the histogram
img = ds.tf.set_background(ds.tf.shade(agg, how="log", cmap=cc.bmw), "white").to_pil()  # create a rasterized imageplt.imshow(img)
plt.imshow(img)
plt.axis('off')
plt.show()

In [39]:
ptest.rename(columns={'ARR_DELAY':'ARRIVAL_DELAY'}, inplace=True)
ptest.to_csv("test_emiel_v7.csv",index=False)
ptrain.to_csv("train_emiel_v7.csv",index=False)

# Modeling

In [ ]:
#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'


#no.of cores each processor is having
!lscpu | grep 'Core(s) per socket'

!cat /proc/meminfo | grep 'MemAvailable'

In [17]:
# h2o.cluster().shutdown()
h2o.init(max_mem_size = "14G")

In [18]:
htrain = h2o.H2OFrame(ptrain)
if "AIRLINE" in htrain.columns:
    htrain["AIRLINE"] = htrain["AIRLINE"].asfactor()
    htrain
htrain

In [19]:
y = "ARRIVAL_DELAY"
X = htrain.columns
X = list(filter(lambda x: x not in [y, "id"], X))
print(y)
print(X)

# grid search

sucks

In [ ]:
train, valid, test = htrain.split_frame(ratios=[.4,.3], seed=35)

---
# ntrees

Default 50,
results looked promising but actually trying `ntrees = 500` really overfit the data. no dice.

we stick with `ntrees = 150`

In [ ]:
xgb_params_ntrees = {
#     'learn_rate': [0.01, 0.1],
#     'max_depth': [3, 5, 9],
#     'sample_rate': [0.8, 1.0],
#     'col_sample_rate': [0.2, 0.5, 1.0],
#     'skip_drop': [0, 0.5, 1.0],
    'ntrees': [30, 50, 100, 200, 300, 400, 500],
}

# Train and validate a cartesian grid of GBMs
xgb_grid_ntrees = H2OGridSearch(model=H2OXGBoostEstimator,
                          grid_id='xgb_grid_ntrees',
                          hyper_params=xgb_params_ntrees)
xgb_grid_ntrees.train(
    x=X, y=y,
    training_frame=train,
    validation_frame=valid,
    seed=35
)


In [ ]:
# Get the grid results, sorted by validation RMSE
xgb_grid_ntrees_perf = xgb_grid_ntrees.get_grid(sort_by='RMSE', decreasing=False)
xgb_grid_ntrees_perf

In [ ]:
# Now let's evaluate the model performance on a test set
for i, m in enumerate(xgb_grid_ntrees_perf.models):
    print(f' #{i} rmse: {m.model_performance(test).rmse()}')

---
## skipdrop

did absolutetly nothing

probably because we did not set `dart` as the booster haha

In [ ]:
# Now let's evaluate the model performance on a test set
for i, m in enumerate(xgb_grid_skipdrop_perf.models):
    print(f' #{i} rmse: {m.model_performance(test).rmse()}')

---
## max depth

In [ ]:
xgb_params_maxd = {
    'max_depth': [3, 5, 9],
}

# Train and validate a cartesian grid of GBMs
xgb_grid_maxd = H2OGridSearch(model=H2OXGBoostEstimator,
                          grid_id='xgb_grid_maxd',
                          hyper_params=xgb_params_maxd)
xgb_grid_maxd.train(
    x=X, y=y,
    training_frame=train,
    validation_frame=valid,
    seed=35
)

In [ ]:
# Get the grid results, sorted by validation RMSE
xgb_grid_maxd_perf = xgb_grid_maxd.get_grid(sort_by='RMSE', decreasing=False)
xgb_grid_maxd_perf

In [ ]:
# Now let's evaluate the model performance on a test set
for i, m in enumerate(xgb_grid_maxd_perf.models):
    print(f' #{i} rmse: {m.model_performance(test).rmse()}')

---
# model training

In [ ]:
train, valid = htrain.split_frame(ratios=[.8], seed=35)

In [20]:
%%time

# Build and train the model:
flight_xgb3 = H2OXGBoostEstimator(
    booster            ='dart',
    normalize_type     ="tree",
    seed               = 35,
    ntrees             = 200,
    stopping_metric    = "RMSE",
    stopping_rounds    = 100,
    stopping_tolerance = 0.01,
)

flight_xgb3.train(
    x = X, y = y,
    training_frame     = htrain
)

In [ ]:
h2o.save_model(model=flight_xgb3, path="", force=True)

In [ ]:
flight_xgb3.model_performance(valid)

# Predicting

In [21]:
htest = h2o.H2OFrame(ptest)
htest["AIRLINE"] = htest["AIRLINE"].asfactor()
htest

In [22]:
pred = flight_xgb3.predict(htest)
pred

In [24]:
pred_df = pred.as_data_frame()
kaggle = pd.DataFrame()

kaggle["id"] = ptest["id"]
kaggle["ARRIVAL_DELAY"] = pred_df["predict"]
kaggle = kaggle.sort_values(by="id")
kaggle.to_csv("xgb7_emiel_new_features.csv",index=False)

In [ ]:
compare = flights_train["ARRIVAL_DELAY"].to_frame().describe()
compare["PREDICTIONS"] = pred_df.describe()
compare["DIFF"] = compare["PREDICTIONS"] - compare["ARRIVAL_DELAY"]
compare["DIFF"]["count"] = "-"
compare

--- 
# Is CatBoost the coolest thing ever?

In [ ]:
# who knows
# also yes

import catboost as cb

In [ ]:
cbr = cb.CatBoostRegressor(loss_function="RMSE")
cbr.fit(X_train_val, y_train_val)
cbr.predict(X_test)
eval_pred = cbr.predict(X_test)
pd.DataFrame(eval_pred, columns=['ARRIVAL_DELAY']).to_csv("flight_result.csv", index_label='id')

---
# another approach

will adding up all delay column give you the perfect score?

computer says: `no`

In [35]:
ptest["predi"] = ptest.CARRIER_DELAY + ptest.WEATHER_DELAY + ptest.NAS_DELAY + ptest.SECURITY_DELAY + ptest.LATE_AIRCRAFT_DELAY

In [36]:
ptest["error"] = ptest.predi - ptest.ARR_DELAY

In [38]:
((ptest.error ** 2).sum())/ptest.shape[0]

---
<br>
<br>
<br>
<br>
<br>
<br>
-> EOF